In [2]:
import os
from dotenv import load_dotenv
from pinecone import Pinecone
from sentence_transformers import SentenceTransformer
import re
from typing import List, Dict, Optional, Tuple

# 환경변수 로드
load_dotenv()

class PerfumeQueryTester:
    def __init__(self):
        # Pinecone 연결
        self.pc = Pinecone(api_key=os.getenv('PINECONE_API_KEY'))
        self.index = self.pc.Index("perfume-search")
        
        # 임베딩 모델 로드
        self.model = SentenceTransformer('jhgan/ko-sroberta-multitask')
        
    def extract_brand_from_query(self, query: str) -> Optional[str]:
        """쿼리에서 브랜드명 추출"""
        brands = ['샤넬', '딥디크', '크리드', '톰포드', '톰 포드', '조malons', '버버리', 
                 '불가리', '구찌', '프라다', '에르메스', '이솝', '켄조', '랑콤',
                 'chanel', 'creed', 'tom ford', 'jo malone', 'burberry', 
                 'bulgari', 'gucci', 'prada', 'hermes', 'aesop']
        
        query_lower = query.lower()
        for brand in brands:
            if brand.lower() in query_lower:
                return brand
        return None

    def extract_perfume_name(self, query: str) -> Optional[str]:
        """쿼리에서 향수명 추출"""
        perfume_patterns = [
            r'넘버\s*5|number\s*5|no\s*5',
            r'어벤투스|aventus',
            r'블랙\s*오피움|black\s*opium',
            r'사바주|sauvage',
        ]
        
        query_lower = query.lower()
        for pattern in perfume_patterns:
            match = re.search(pattern, query_lower)
            if match:
                return match.group()
        return None

    def build_metadata_filter(self, query: str) -> Dict:
        """쿼리 분석해서 메타데이터 필터 구성"""
        filters = {}
        
        # 브랜드 필터
        brand = self.extract_brand_from_query(query)
        if brand:
            filters['brand'] = {'$eq': brand}
        
        # 향수명이 특정되면 name 필터도 추가
        perfume_name = self.extract_perfume_name(query)
        if perfume_name:
            # 부분 매칭을 위해 contains 같은 기능이 없으므로 임베딩 검색에 의존
            pass
        
        return filters

    def search_perfumes(self, query: str, top_k: int = 5) -> List[Dict]:
        """향수 검색"""
        print(f"\n🔍 검색 쿼리: '{query}'")
        
        # 메타데이터 필터 구성
        metadata_filter = self.build_metadata_filter(query)
        if metadata_filter:
            print(f"📋 적용된 필터: {metadata_filter}")
        
        # 쿼리 임베딩 생성
        query_embedding = self.model.encode(query).tolist()
        
        # Pinecone에서 검색
        try:
            search_results = self.index.query(
                vector=query_embedding,
                filter=metadata_filter if metadata_filter else None,
                top_k=top_k,
                include_metadata=True
            )
            
            results = []
            for match in search_results['matches']:
                result = {
                    'score': match['score'],
                    'brand': match['metadata'].get('brand', ''),
                    'name': match['metadata'].get('name', ''),
                    'eng_name': match['metadata'].get('eng_name', ''),
                    'size_ml': match['metadata'].get('size_ml', 0),
                    'price_krw': match['metadata'].get('price_krw', 0),
                    'concentration': match['metadata'].get('concentration', ''),
                    'gender': match['metadata'].get('gender', ''),
                    'main_notes': match['metadata'].get('main_notes', []),
                    'best_season': match['metadata'].get('best_season', ''),
                    'best_time': match['metadata'].get('best_time', ''),
                    'detail_url': match['metadata'].get('detail_url', '')
                }
                results.append(result)
            
            return results
            
        except Exception as e:
            print(f"❌ 검색 중 오류: {e}")
            return []

    def format_results(self, results: List[Dict]) -> None:
        """검색 결과 출력"""
        if not results:
            print("❌ 검색 결과가 없습니다.\n")
            return
        
        print(f"✅ 검색 결과 ({len(results)}개):")
        print("-" * 60)
        
        for i, result in enumerate(results, 1):
            print(f"{i}. {result['brand']} - {result['name']}")
            print(f"   영문명: {result['eng_name']}")
            print(f"   용량/가격: {result['size_ml']}ml / {result['price_krw']:,}원")
            print(f"   농도: {result['concentration']}")
            print(f"   성별: {result['gender']}")
            
            if result['main_notes']:
                print(f"   주요향: {', '.join(result['main_notes'])}")
            
            if result['best_season']:
                print(f"   추천시즌: {result['best_season']}")
                
            if result['best_time']:
                print(f"   추천시간: {result['best_time']}")
            
            print(f"   유사도 점수: {result['score']:.4f}")
            print(f"   상세정보: {result['detail_url']}")
            print()

    def run_example_queries(self):
        """예시 쿼리들 실행"""
        example_queries = [
            "레전더리 미츠코 오 드 퍼퓸 어코드좀 알려줘",
            "길티 앱솔루트 뿌르 옴므 오 드 퍼퓸 노트좀 알려줘?",
            "엉브레 오 드 퍼퓸 향수 노트좀 알려줘",
        
        ]
        
        print("="*70)
        print("🌸 향수 검색 시스템 테스트")
        print("="*70)
        
        for query in example_queries:
            results = self.search_perfumes(query, top_k=3)
            self.format_results(results)
            print("="*70)

    def interactive_search(self):
        """대화형 검색"""
        print("🌸 향수 검색 시스템 (종료하려면 'quit' 입력)")
        print("-" * 50)
        
        while True:
            query = input("\n검색어를 입력하세요: ").strip()
            
            if query.lower() in ['quit', 'exit', '종료', 'q']:
                print("검색을 종료합니다.")
                break
                
            if not query:
                continue
                
            results = self.search_perfumes(query, top_k=5)
            self.format_results(results)

def main():
    """메인 실행 함수"""
    tester = PerfumeQueryTester()
    
    print("어떤 모드로 실행하시겠습니까?")
    print("1. 예시 쿼리 자동 실행")
    print("2. 대화형 검색")
    
    choice = input("선택하세요 (1 또는 2): ").strip()
    
    if choice == "1":
        tester.run_example_queries()
    elif choice == "2":
        tester.interactive_search()
    else:
        print("잘못된 선택입니다. 예시 쿼리를 실행합니다.")
        tester.run_example_queries()

if __name__ == "__main__":
    main()

어떤 모드로 실행하시겠습니까?
1. 예시 쿼리 자동 실행
2. 대화형 검색
🌸 향수 검색 시스템 테스트

🔍 검색 쿼리: '레전더리 미츠코 오 드 퍼퓸 어코드좀 알려줘'
✅ 검색 결과 (3개):
------------------------------------------------------------
1. 겔랑 - 레전더리 미츠코 오 드 퍼퓸
   영문명: Legendary Mitsuko
   용량/가격: 75.0ml / 170,300.0원
   농도: 오 드 퍼퓸
   성별: Female
   주요향: woody, spicy, floral, earthy, rose
   추천시즌: 겨울
   추천시간: 올데이
   유사도 점수: 0.5860
   상세정보: https://www.bysuco.com/product/show/140974

2. 톰 포드 - 미르 미스테르 오 드 퍼퓸
   영문명: Myrrhe Myst?re
   용량/가격: 30.0ml / 239,000.0원
   농도: 오 드 퍼퓸
   성별: Unisex
   주요향: woody, aromatic, citrus, lavender, musky
   추천시즌: 겨울
   추천시간: 야간용
   유사도 점수: 0.5751
   상세정보: https://www.bysuco.com/product/show/225713

3. 톰 포드 - 미르 미스테르 오 드 퍼퓸
   영문명: Myrrhe Myst?re
   용량/가격: 50.0ml / 353,200.0원
   농도: 오 드 퍼퓸
   성별: Unisex
   주요향: woody, aromatic, citrus, lavender, musky
   추천시즌: 겨울
   추천시간: 야간용
   유사도 점수: 0.5751
   상세정보: https://www.bysuco.com/product/show/225713


🔍 검색 쿼리: '길티 앱솔루트 뿌르 옴므 오 드 퍼퓸 노트좀 알려줘?'
✅ 검색 결과 (3개):
-------------------------